In [ ]:
#wd<-"/home/jovyan/SCI_project/result/SPOTlight/WT/"
wd<-"/home/jovyan/SCI_project/result/RCTD/WT/"
setwd(wd)
getwd()
#result_dir<-"quantation/"
result_dir<-"interaction/"
dir.create(result_dir)

In [ ]:
library(reshape)
library(Seurat)
library(ggplot2)
library(patchwork)
library(dplyr)
library(hdf5r)
library(png)
library(ggalluvial)
#library(tidyverse)
library(clustree)
library(RCTD)
library(Matrix)
library(SPOTlight)
library(future)
library(grid)
library(RColorBrewer)
library(pheatmap)
library(circlize)
library(igraph)
library(ComplexHeatmap)
library(stringr)
library(ComplexHeatmap)
library(cowplot)

In [ ]:
#da<-readRDS("/home/jovyan/SCI_project/result/RCTD/KO/KO.merge.RCTD.ad.rds")
da<-readRDS("/home/jovyan/SCI_project/result/Seurat/WT_KO.merge/WT.merge.RCTD.ad.rds")
#da<-readRDS("/home/jovyan/SCI_project/result/SPOTlight/WT/WT.merge.SPOTlight.ad.rds")
da

In [5]:
### first preview domain cell type change using js annotated domains to give zq interested confirm list
da$orig.ident<-factor(da$orig.ident,levels = c('WT_sham_H_R2_1mm','WT_sham_H_R2','WT_sham_T_210323','WT_sham_T_R2_1mm',
                                              'WT_3h_H_R2_1mm','WT_3h_H_R2','WT_3h_T_R2','WT_3h_T_210330_1mm',
                                              'WT_24h_H_R1_1mm','WT_24h_H_R1','WT_24h_T_201231','WT_24h_T_R1_1mm',
                                              'WT_72h_H_R1_1mm','WT_72h_H_201231','WT_72h_T_201231','WT_72h_T_R1_1mm')
                     #c('Sarm1_KO_sham_H_R1_1mm','Sarm1_KO_sham_H_210323','Sarm1_KO_sham_T_R1_0.5mm','Sarm1_KO_sham_T_R1_1mm',
                     #  'Sarm1_KO_3h_H_R1_1mm','Sarm1_KO_3h_H_R1_0.5mm','Sarm1_KO_3h_T_R1_0.5mm','Sarm1_KO_3h_T_R1_1mm',
                     #  'Sarm1_KO_24h_H_R1_1mm','Sarm1_KO_24h_H_201231','Sarm1_KO_24h_T_201231','Sarm1_KO_24h_T_1mm_201231',
                     #  'Sarm1_KO_72h_H_R1_1mm','Sarm1_KO_72h_H_210323','Sarm1_KO_72h_T_210323','Sarm1_KO_72h_T_R1_1mm')
                     )
da$timming<-factor(da$timming,levels = c('WT_sham','WT_3h','WT_24h','WT_72h')
                  #c('Sarm1_KO_sham','Sarm1_KO_3h','Sarm1_KO_24h','Sarm1_KO_72h')
                  )

In [6]:
da$distance<-NA
da@meta.data[grepl("1mm",da@meta.data$orig.ident,fixed = T),]$distance<-"1mm"
da@meta.data[!grepl("1mm",da@meta.data$orig.ident,fixed = T),]$distance<-"0.5mm"

In [ ]:
da$mgroup<-paste(da$timming,da$distance,sep="_")
unique(da$mgroup)

In [8]:
da$mgroup<-factor(da$mgroup,levels = c('WT_sham_0.5mm',
                                       'WT_sham_1mm',
                                       'WT_3h_0.5mm',
                                       'WT_3h_1mm',
                                       'WT_24h_0.5mm',
                                       'WT_24h_1mm',
                                       'WT_72h_0.5mm',
                                       'WT_72h_1mm')
                 #c('Sarm1_KO_sham_0.5mm',
                 #  'Sarm1_KO_sham_1mm',
                 #  'Sarm1_KO_3h_0.5mm',
                 #  'Sarm1_KO_3h_1mm',
                 #  'Sarm1_KO_24h_0.5mm',
                 #  'Sarm1_KO_24h_1mm',
                 #  'Sarm1_KO_72h_0.5mm',
                 #  'Sarm1_KO_72h_1mm'
                 #  )
                 )

In [ ]:
me<-da@meta.data
dim(me)
head(me)

In [ ]:
cell_types_multi<-c('Astrocyte','Dendritic','Div.Myeloid','Endothelial',
                    'Ependymal','Fibroblast','Lymphocyte','Macrophage',
                    'Microglia','Monocyte','Neuron','Neutrophil',
                    'Oligodendrocyte','OPC','Pericyte')
cell_types_multi

In [12]:
immune_cell_types<-c('Dendritic','Div.Myeloid','Lymphocyte','Macrophage',
                    'Microglia','Monocyte','Neutrophil')
vascular_cell_types<-c('Fibroblast','Endothelial','Pericyte')
macroglia_cell_types<-c('Astrocyte','Ependymal',
                    'Oligodendrocyte','OPC')

In [ ]:
#1.dotplot extract domains spots
wm<-da[rownames(da),rownames(me[me$domains=="WM",])]
dim(wm)
mg<-da[rownames(da),rownames(me[me$domains=="MG",])]
dim(mg)
dh<-da[rownames(da),rownames(me[me$domains=="DH",])]
dim(dh)
vh<-da[rownames(da),rownames(me[me$domains=="VH",])]
dim(vh)

In [ ]:
#source("/home/jovyan/SCI_project/scripts/RCTD/DotPlot_ct.R")
p3 <- DotPlot(wm, features = cell_types_multi, group.by = 'mgroup', scale = F) + 
      theme(axis.text.x = element_text(angle = 90, size = 8, margin = margin(55,0,0,0)))
options(repr.plot.width = 12, repr.plot.height = 7)
p3+ggtitle("WM")
ggsave(paste0(result_dir,"WM_dotplot.distance.png"),width = 12,height = 7,dpi = 300)
#pdf("WT.sham.merge_dotPlot_deconvByTimming_RCTD_integrated_scale.data_pct0.7.pdf", width = 15, height = 7)
#p3
#dev.off()

In [ ]:
p3 <- DotPlot(wm, features = immune_cell_types, group.by = 'mgroup', scale = F) + 
      theme(axis.text.x = element_text(angle = 90, size = 8, margin = margin(55,0,0,0)))
options(repr.plot.width = 8, repr.plot.height = 7)
p3+ggtitle("WM")
#ggsave(paste0(result_dir,"WM_dotplot.distance.png"),width = 12,height = 7,dpi = 300)

In [ ]:
p3 <- DotPlot(mg, features = cell_types_multi, group.by = 'mgroup', scale = F) + 
      theme(axis.text.x = element_text(angle = 90, size = 8, margin = margin(55,0,0,0)))
options(repr.plot.width = 12, repr.plot.height = 7)
p3+ggtitle("MG")
ggsave(paste0(result_dir,"MG_dotplot.distance.png"),width = 12,height = 7,dpi = 300)

In [ ]:
p3 <- DotPlot(vh, features = cell_types_multi, group.by = 'mgroup', scale = F) + 
      theme(axis.text.x = element_text(angle = 90, size = 8, margin = margin(55,0,0,0)))
options(repr.plot.width = 12, repr.plot.height = 7)
p3+ggtitle("VH")
ggsave(paste0(result_dir,"VH_dotplot.distance.png"),width = 12,height = 7,dpi = 300)

In [ ]:
p3 <- DotPlot(dh, features = cell_types_multi, group.by = 'mgroup', scale = F) + 
      theme(axis.text.x = element_text(angle = 90, size = 8, margin = margin(55,0,0,0)))
options(repr.plot.width = 12, repr.plot.height = 7)
p3+ggtitle("DH")
ggsave(paste0(result_dir,"DH_dotplot.distance.png"),width = 12,height = 7,dpi = 300)

In [367]:
#2.stacked bar chart
wmme<-me[me$domains=="WM",]
mgme<-me[me$domains=="MG",]
vhme<-me[me$domains=="VH",]
dhme<-me[me$domains=="DH",]

In [ ]:
colnames(wmme)

In [ ]:
temp<-wmme[,c("mgroup",cell_types_multi)]
head(temp)

In [ ]:

mtemp<-melt(temp,id=c("mgroup"))

#mtemp$orig.ident<-factor(mtemp$orig.ident,levels=c('WT_sham_H_R2_1mm','WT_sham_H_R2','WT_sham_T_210323','WT_sham_T_R2_1mm',
#                                              'WT_3h_H_R2_1mm','WT_3h_H_R2','WT_3h_T_R2','WT_3h_T_210330_1mm',
#                                              'WT_24h_H_R1_1mm','WT_24h_H_R1','WT_24h_T_201231','WT_24h_T_R1_1mm',
#                                              'WT_72h_H_R1_1mm','WT_72h_H_201231','WT_72h_T_201231','WT_72h_T_R1_1mm'))
#mtemp$timming<-factor(mtemp$timming,levels =c('Sarm1_KO_sham','Sarm1_KO_3h','Sarm1_KO_24h','Sarm1_KO_72h')# c('WT_sham','WT_3h','WT_24h','WT_72h'))
mtemp$mgroup<-factor(mtemp$mgroup,levels = c('WT_sham_0.5mm',
                                       'WT_sham_1mm',
                                       'WT_3h_0.5mm',
                                       'WT_3h_1mm',
                                       'WT_24h_0.5mm',
                                       'WT_24h_1mm',
                                       'WT_72h_0.5mm',
                                       'WT_72h_1mm')
                   # c('Sarm1_KO_sham_0.5mm',
                   #'Sarm1_KO_sham_1mm',
                   #'Sarm1_KO_3h_0.5mm',
                   #'Sarm1_KO_3h_1mm',
                   #'Sarm1_KO_24h_0.5mm',
                   #'Sarm1_KO_24h_1mm',
                   #'Sarm1_KO_72h_0.5mm',
                   #'Sarm1_KO_72h_1mm')
                    )
colnames(mtemp)<-c("time_distance","cell_type","proportion")
head(mtemp)


In [404]:
group<-"time_distance"
domain<-"WM"

In [ ]:
### probability distribution of each cell type across all sections 2021.12.8
meta<-da@meta.data
colnames(meta)
meta<-meta[,c(21:ncol(meta))]
colnames(meta)

In [ ]:
#meta<-as.data.frame(t(meta))
#meta$cell_type<-rownames(meta)
melt_meta<-melt(meta,id=c('cell_type'))
head(melt_meta)

In [42]:
library(ggridges)

In [ ]:
#remove 0 row for better visualization
options(repr.plot.width=5,repr.plot.height=5)
p<-melt_meta %>% 
    filter(value!=0) %>%
    ggplot(aes(x=value,y=cell_type,fill=cell_type))+
        geom_density_ridges()+
        theme_ridges() + 
      theme(legend.position = "none")

p+geom_vline(xintercept=0.1)
ggsave("WT.merge.RCTD.celltype_ridge-density.cutoff01.png",width =5,height=5,dpi = 300 )

In [ ]:
cell_types_multi<-colnames(da@meta.data[,c(21:ncol(da@meta.data))])
cell_types_multi

In [ ]:
temp<-melt_meta[melt_meta$cell_type==cell_types_multi[11] & melt_meta$value!=0,]
dim(temp)

In [93]:
temp<-da@meta.data[,c(21:ncol(da@meta.data))]
temp[temp<0.35]<-0

In [94]:
da@meta.data[,c(21:ncol(da@meta.data))]<-temp

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 18,resolution=300)
p3 <- SpatialFeaturePlot(da, features = "Oligodendrocyte", combine = FALSE)
    do.call(ggpubr::ggarrange, c(p3, list(ncol = 4, nrow = 4, common.legend = T, legend = "right")))
#ggsave("WT.merge.RCTD.celltype.cutoff035.spatial.png",width =20,height=18,dpi = 300)

In [ ]:
#dir.create("cutoff035/")

for(i in cell_types_multi){
    options(repr.plot.width = 20, repr.plot.height = 18,resolution=300)
    p3 <- SpatialFeaturePlot(da, features = i, combine = FALSE)
        do.call(ggpubr::ggarrange, c(p3, list(ncol = 4, nrow = 4, common.legend = T, legend = "right")))
    ggsave(paste0("cutoff035/WT.merge.RCTD.",i,"celltype.cutoff035.spatial.png"),width =20,height=18,dpi = 300)
}
